**5. VISTAS**

In [ ]:
USE bichos_de_bolsillo

GO

Create VIEW vw_TablaBichos
as
    SELECT bicho_id as 'ID', bicho_nombre as 'Bicho'
    from Bichos

GO

Create VIEW vw_UsuariosResgistrados
as
    SELECT usuario_nombre as 'Usuarios', usuario_correo AS 'Correo'
    from Usuarios

GO


CREATE VIEW vw_Recolecciones
as
    SELECT (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = recoleccion_usuario) as 'Usuario',
        (SELECT bicho_nombre
        FROM Bichos
        WHERE bicho_id = recoleccion_bicho) AS 'Bicho',
        recoleccion_nivel as 'Nivel',
        recoleccion_fecha as 'Fecha de Recoleccion'
    from Recolecciones

GO

CREATE VIEW vw_Combates
as
    SELECT
        combate_id as 'ID',
        combate_fecha as 'Fecha',
        (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = usuario_uno) as 'Jugador 1',
        (SELECT usuario_nombre
        FROM Usuarios
        WHERE usuario_id = usuario_dos) as 'Jugador 2',
        (SELECT count(combate_id)
        FROM Rondas
        WHERE combate_id = Combates.combate_id and ganador = usuario_uno) AS 'Ganadas del jugador 1',
        (SELECT count(combate_id)
        FROM Rondas
        WHERE combate_id = Combates.combate_id and ganador = usuario_dos) AS 'Ganadas del jugador 2'
    FROM Combates

GO

SELECT *
from vw_TablaBichos
SELECT *
from vw_UsuariosResgistrados
SELECT *
from vw_Recolecciones
SELECT *
from vw_Combates


**6. FUNCIONES**

In [ ]:
CREATE FUNCTION dbo.GetRondas(@combate_id int)
RETURNS TABLE 
AS 
RETURN 
(
    SELECT
    ronda_noRonda AS 'No. de ronda',
    (SELECT bicho_nombre
    FROM Bichos
    WHERE bicho_id = bicho_uno) AS 'Bicho 1',
    (SELECT bicho_nombre
    FROM Bichos
    WHERE bicho_id = bicho_dos) AS 'Bicho 2',
    (SELECT catalogo_nombre
    FROM Catalogo_Ataques
    WHERE catalogo_id = ataque_uno) AS 'Ataque 1',
    (SELECT catalogo_nombre
    FROM Catalogo_Ataques
    WHERE catalogo_id = ataque_dos) AS 'Ataque 2',
    nivel_daño_uno AS 'Daño 1',
    nivel_daño_dos AS 'Daño 2',
    ganador AS 'Ganador'
FROM Rondas
WHERE combate_id = @combate_id
);

GO

CREATE FUNCTION dbo.GetRondasPerPlayer(@combate_id int, @usuario int)
RETURNS INT 
AS 
BEGIN
    RETURN (
SELECT count(combate_id)
    FROM Rondas
    WHERE combate_id = @combate_id AND ganador = @usuario
)
END;

GO

CREATE FUNCTION dbo.GetGanadorCombate(@combate_id int, @usuario_uno int, @usuario_dos int)
RETURNS INT 
AS 
BEGIN
    DECLARE @ganados1 int, @ganados2 int;

    SET @ganados1 = dbo.GetRondasPerPlayer(@combate_id, @usuario_uno)

    SET @ganados2 = dbo.GetRondasPerPlayer(@combate_id, @usuario_uno)

    IF(@ganados1 > @ganados2)
        RETURN(@ganados1)
    ELSE IF(@ganados1 = @ganados2)
        RETURN(0)
    RETURN(@ganados2)
END;

GO


    Probar funciones

In [ ]:
SELECT *
FROM dbo.GetRondas(1)
SELECT dbo.GetRondasPerPlayer(1, 4) AS 'jugador 1'
SELECT dbo.GetRondasPerPlayer(1, 3) AS 'jugador 2'
SELECT dbo.GetGanadorCombate(1, 4, 3) AS 'Ganador';